# Making a SQLite databased combining multiple tables

In [ ]:
from pyprojroot import here
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
import pandas as pd
from pathlib import Path

data_dir = Path(str(here("Data")))
print( f"Data directory: {data_dir}")


applications_file_name = "applications-energy.csv"
file_path = data_dir / applications_file_name
applications_df = pd.read_csv(file_path)
print( f"Application file path: {file_path}")

comments_file_name = "comments-energy.csv"
file_path = data_dir / comments_file_name
comments_df = pd.read_csv(file_path)
print( f"Comments file path: {file_path}")


compositions_file_name = "compositions-energy.csv"
file_path = data_dir / compositions_file_name
compositions_df = pd.read_csv(file_path)
print( f"Composition file path: {file_path}")

melting_points_file_name = "melting-points-energy.csv"
file_path = data_dir / melting_points_file_name
melting_points_df = pd.read_csv(file_path)
print( f"Melting points file path: {file_path}")


db_path = str(here("Data")) + "/ICDD-energy.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)

comments_df.to_sql("Comments", engine, index=False)
applications_df.to_sql("Applications", engine, index=False)
compositions_df.to_sql("Compositions", engine, index=False)
melting_points_df.to_sql("Melting Points", engine, index=False)

Data directory: /Users/arashkhajeh/GitHub/LLM-for-Materials/Data
Application file path: /Users/arashkhajeh/GitHub/LLM-for-Materials/Data/applications-energy.csv
comments file path: /Users/arashkhajeh/GitHub/LLM-for-Materials/Data/comments-energy.csv
Composition file path: /Users/arashkhajeh/GitHub/LLM-for-Materials/Data/compositions-energy.csv
Melting points file path: /Users/arashkhajeh/GitHub/LLM-for-Materials/Data/melting-points-energy.csv


525

In [9]:
# validate the connection to the vectordb
from langchain_community.utilities import SQLDatabase
from pyprojroot import here

db_path = str(here("Data")) + "/ICDD-energy.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM comments LIMIT 10;")

sqlite
['Applications', 'Comments', 'Compositions', 'Melting Points']


"[(353, 10359, None, None, 'Decomposes at 1750 C'), (494, 10503, None, None, None), (569, 10579, None, None, None), (618, 10628, None, None, 'Transition to calcite at 520. Antacid'), (636, 10646, None, 'A new method of X-Ray crystal analysis', None), (667, 10677, None, None, None), (696, 10706, None, None, None), (738, 10750, None, None, None), (780, 10792, None, None, None), (784, 10796, None, None, None)]"

In [10]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())

# Optional: print model name if you set it
print("Model name:", os.getenv("gpt_deployment_name"))  # e.g., "gpt-3.5-turbo"

Environment variables are loaded: True
Model name: gpt-3.5-turbo


In [22]:
from openai import OpenAI

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",#os.getenv("gpt_deployment_name"),
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [2]:
# SQL Query chain

# Load the LLM
from langchain.chat_models import ChatOpenAI

model_name = os.getenv("gpt_deployment_name")
openai_api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(model_name=model_name,
                 temperature=0,
                 openai_api_key = openai_api_key)

NameError: name 'os' is not defined

In [25]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "What are the table names in the database"})
print(response)

SELECT


In [21]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many entries have melting point?"})
print(response)

SELECT


In [17]:
# execute the query
db.run(response)

'[(2,)]'

In [21]:
# Using Chain

from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many entries have melting point?"})

'[(2,)]'

In [22]:
# Answer the question in a user friendly manner
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many entries have melting point?"})

'There are 2 entries in the comments table that have a melting point value.'

# Using SQL Agent

In [28]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("gpt_deployment_name"))

Environment variables are loaded: True
test by reading a variable: gpt-3.5-turbo


In [29]:
# Load the LLM
from langchain.chat_models import ChatOpenAI

model_name = os.getenv("gpt_deployment_name")
openai_api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(model_name=model_name,
                 temperature=0,
                 openai_api_key = openai_api_key)

In [30]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [32]:
response = agent_executor.invoke({"input": "Describe the comments table."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


comments
Invoking: `sql_db_schema` with `{'table_names': 'comments'}`



CREATE TABLE comments (
	"Unnamed: 0" BIGINT, 
	"ProductID" BIGINT, 
	"Melting Point" TEXT, 
	"Article Title" TEXT
)

/*
3 rows from comments table:
Unnamed: 0	ProductID	Melting Point	Article Title
0	40203804	None	Crystal structures and magnetic properties of the BaAl4-type structure derivatives in Gd-Al-Ga syste
1	40017624	None	Structural Features and Magnetic Properties of Solid Solutions in the CuFe2O4-Li0.5Fe2.5O4-Mn3O4 Sys
2	40231825	None	Crystal structure, thermal expansion and high-temperature electrical conductivity of A-site deficien
*/The comments table has the following columns:
1. Unnamed: 0 (BIGINT)
2. ProductID (BIGINT)
3. Melting Point (TEXT)
4. Article Title (TEXT)

Here are 3 sample rows from the comments table:
1. Unnamed: 0: 40203804, ProductID: 40203804, Melting Point: None, Article Title: Crystal structure

In [35]:
print(response["output"])

The comments table has the following columns:
1. Unnamed: 0 (BIGINT)
2. ProductID (BIGINT)
3. Melting Point (TEXT)
4. Article Title (TEXT)

Here are 3 sample rows from the comments table:
1. Unnamed: 0: 40203804, ProductID: 40203804, Melting Point: None, Article Title: Crystal structures and magnetic properties of the BaAl4-type structure derivatives in Gd-Al-Ga system
2. Unnamed: 0: 40017624, ProductID: 40017624, Melting Point: None, Article Title: Structural Features and Magnetic Properties of Solid Solutions in the CuFe2O4-Li0.5Fe2.5O4-Mn3O4 Sys
3. Unnamed: 0: 40231825, ProductID: 40231825, Melting Point: None, Article Title: Crystal structure, thermal expansion and high-temperature electrical conductivity of A-site deficien


In [39]:
response = agent_executor.invoke({"input": "How many entries have melting points in the comments table?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


comments
Invoking: `sql_db_schema` with `{'table_names': 'comments'}`



CREATE TABLE comments (
	"Unnamed: 0" BIGINT, 
	"ProductID" BIGINT, 
	"Melting Point" TEXT, 
	"Article Title" TEXT
)

/*
3 rows from comments table:
Unnamed: 0	ProductID	Melting Point	Article Title
0	40203804	None	Crystal structures and magnetic properties of the BaAl4-type structure derivatives in Gd-Al-Ga syste
1	40017624	None	Structural Features and Magnetic Properties of Solid Solutions in the CuFe2O4-Li0.5Fe2.5O4-Mn3O4 Sys
2	40231825	None	Crystal structure, thermal expansion and high-temperature electrical conductivity of A-site deficien
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM comments WHERE "Melting Point" IS NOT NULL'}`


[(2,)]There are 2 entries in the comments table that have melting points specified.

> Finished chain.


In [42]:
response = agent_executor.invoke({"input": "Can you tell me in what article in the comments table CuFe2O4 is appeared? Also show me th product ID of this entry"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


comments
Invoking: `sql_db_schema` with `{'table_names': 'comments'}`



CREATE TABLE comments (
	"Unnamed: 0" BIGINT, 
	"ProductID" BIGINT, 
	"Melting Point" TEXT, 
	"Article Title" TEXT
)

/*
3 rows from comments table:
Unnamed: 0	ProductID	Melting Point	Article Title
0	40203804	None	Crystal structures and magnetic properties of the BaAl4-type structure derivatives in Gd-Al-Ga syste
1	40017624	None	Structural Features and Magnetic Properties of Solid Solutions in the CuFe2O4-Li0.5Fe2.5O4-Mn3O4 Sys
2	40231825	None	Crystal structure, thermal expansion and high-temperature electrical conductivity of A-site deficien
*/
Invoking: `sql_db_query` with `{'query': 'SELECT * FROM comments WHERE "Article Title" LIKE \'%CuFe2O4%\''}`


[(1, 40017624, None, 'Structural Features and Magnetic Properties of Solid Solutions in the CuFe2O4-Li0.5Fe2.5O4-Mn3O4 System')]The article "Structural Features and Magnetic P

In [44]:
print(response["output"])

The article "Structural Features and Magnetic Properties of Solid Solutions in the CuFe2O4-Li0.5Fe2.5O4-Mn3O4 System" contains the mention of CuFe2O4 in the comments table. The product ID of this entry is 40017624.


# Testing on the entire comments dataset

In [45]:
from pyprojroot import here
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
import pandas as pd

csv_file_path = str(here("Data")) + "/comments.csv"
df = pd.read_csv(csv_file_path)


db_path = str(here("Data")) + "/comments.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
df.to_sql("comments", engine, index=False)

1104137

In [46]:
# validate the connection to the vectordb
from langchain_community.utilities import SQLDatabase
from pyprojroot import here

db_path = str(here("Data")) + "/comments.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM comments LIMIT 10;")

sqlite
['comments']


"[(0, 10001, 'Warning: Unindexed pattern. Unit Cell Data Source: Powder Diffraction.', 'Unindexed pattern', 'Powder Diffraction', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

In [47]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())

# Optional: print model name if you set it
print("Model name:", os.getenv("gpt_deployment_name"))  # e.g., "gpt-3.5-turbo"

Environment variables are loaded: True
Model name: gpt-3.5-turbo


In [48]:
from openai import OpenAI

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

client = OpenAI()

response = client.chat.completions.create(
    model=os.getenv("gpt_deployment_name"),
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [ ]:
# SQL Query chain

# Load the LLM
from langchain.chat_models import ChatOpenAI

model_name = os.getenv("gpt_deployment_name")
openai_api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(model_name=model_name,
                 temperature=0,
                 openai_api_key = openai_api_key)

In [50]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many entries are there in comments table?"})
print(response)

SELECT


⚠️ Problem: The Chain Can't Handle Large Schemas Well
The reason you're seeing just SELECT when there are many columns is likely because:

create_sql_query_chain internally injects table schema info into the prompt, and

When the schema (i.e., list of columns) is too long, the total token length exceeds the model's context limit or the prompt becomes too noisy,

So the LLM returns an incomplete SQL query, like just SELECT.



✅ Solutions
✅ Option 1: Use a Narrowed Schema (Only Include Relevant Tables)
LangChain allows you to limit which tables are included in the prompt:

python
Copy
Edit
db = SQLDatabase.from_uri(..., include_tables=["comments"])
chain = create_sql_query_chain(llm, db)
This helps cut down schema size sent to the model.

✅ Option 2: Use a More Capable Model (Like gpt-4 or gpt-3.5-turbo-16k)
Larger models with more context length can handle long table schemas better.

python
Copy
Edit
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
Or if you’re using OpenAI tools agent with auto-function-calling, that often performs better for large schemas:

python
Copy
Edit
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,

In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase  # this might vary depending on your LangChain version

# Step 1: Initialize your LLM
llm = ChatOpenAI(temperature=0, model=model_name)  # or "gpt-4"


# Step 2: Create the SQL agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True, handle_parsing_errors=True)

# Step 3: Ask a question
response = agent.invoke({"input": "How many entries have melting points in the comments table?"})
print(response)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: commentsI should query the schema of the comments table to see if there is a column for melting points.
Action: sql_db_schema
Action Input: comments
CREATE TABLE comments (
	"Unnamed: 0" BIGINT, 
	"ProductID" BIGINT, 
	"FullComment" TEXT, 
	"Warning" TEXT, 
	"Unit Cell Data Source" TEXT, 
	"General Comments" TEXT, 
	"Reason O Quality Was Assigned" TEXT, 
	"Deleted Or Rejected By" TEXT, 
	"Color" TEXT, 
	"Melting Point" TEXT, 
	"Water Loss" TEXT, 
	"Structures" TEXT, 
	"Optical Data" TEXT, 
	"Additional Patterns" TEXT, 
	"Unit Cell" TEXT, 
	"Sample Preparation" TEXT, 
	"Sublimation Point" TEXT, 
	"Analysis" TEXT, 
	"Deleted" TEXT, 
	"Boiling Point" TEXT, 
	"Article Title" TEXT, 
	"Note" TEXT, 
	"Additional Diffraction Lines" TEXT, 
	"Opaque Optical Data" TEXT, 
	"Delete" TEXT, 
	"Reflectance" TEXT, 
	"Vickers Hardness Number" TEXT, 
	"Composition" TEXT, 
	"Fluorescence" TEXT, 
	"Bioactivity" TEXT, 
	"C

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

### Switch to GPT-4

In [71]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase  # this might vary depending on your LangChain version

# Step 1: Initialize your LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")  # or "gpt-4"


# Step 2: Create the SQL agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True, handle_parsing_errors=True)

# Step 3: Ask a question
response = agent.invoke({"input": "How many entries are there in the comments table?"})
print(response)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""commentsThe comments table exists in the database. Now I need to construct a query to count the number of entries in the comments table.
Action: sql_db_query_checker
Action Input: "SELECT COUNT(*) FROM comments"SELECT COUNT(*) FROM commentsThe query is correct. Now I can execute it to get the number of entries in the comments table.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM comments"[(1104137,)]I now know the final answer
Final Answer: There are 1,104,137 entries in the comments table.

> Finished chain.
{'input': 'How many entries are there in the comments table?', 'output': 'There are 1,104,137 entries in the comments table.'}


In [72]:
response = agent.invoke({"input": "How many entries have melting points in the comments table?"})
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""commentsThe only table in the database is 'comments'. I should check the schema of this table to see if there is a column related to melting points.
Action: sql_db_schema
Action Input: comments
CREATE TABLE comments (
	"Unnamed: 0" BIGINT, 
	"ProductID" BIGINT, 
	"FullComment" TEXT, 
	"Warning" TEXT, 
	"Unit Cell Data Source" TEXT, 
	"General Comments" TEXT, 
	"Reason O Quality Was Assigned" TEXT, 
	"Deleted Or Rejected By" TEXT, 
	"Color" TEXT, 
	"Melting Point" TEXT, 
	"Water Loss" TEXT, 
	"Structures" TEXT, 
	"Optical Data" TEXT, 
	"Additional Patterns" TEXT, 
	"Unit Cell" TEXT, 
	"Sample Preparation" TEXT, 
	"Sublimation Point" TEXT, 
	"Analysis" TEXT, 
	"Deleted" TEXT, 
	"Boiling Point" TEXT, 
	"Article Title" TEXT, 
	"Note" TEXT, 
	"Additional Diffraction Lines" TEXT, 
	"Opaque Optical Data" TEXT, 
	"Delete" TEXT, 
	"Reflectance" TEXT, 
	"Vickers Hardness Number" TEXT, 
	"Composition" TEXT, 
	